# Finite Elemente Lösung Bohrpfahl

In [ ]:
from fem import *
from pile import *
from math import *
import numpy as np
from objdict import ObjDict
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

## Parameter

In [ ]:
p = ObjDict()

# Ausgangswerte
p.E = 35e9
p.d = 0.8
p.l = 20
p.F = 2e6
p.rho = 2500
p.g = 9.81
p.C = 1.75e7 * pi * p.d
p.S = 120e6

# Abgeleitet
p.A = pi * p.d**2 / 4
p.EA = p.E * p.A
p.n = p.g * p.rho * p.A

# Ausgeben
print(p)

## Exakte Lösung

In [ ]:
a1 = 3 + 7 * pi
a2 = 7 * pi - 3

def u(x):
    return np.exp(-x / 20) / (3500000 * pi * (a1 * np.exp(2) - a2)) * (
            25000 * (a1 * np.exp(2) + a2 * np.exp(x / 10)) -
            2943 * pi * np.exp(1) * (np.exp(x / 10) + 1) +
            981 * pi * np.exp(x / 20) * (a1 * np.exp(2) - a2) 
        )

def N(x):
    return - 80 * np.exp(-x / 20) / (a1 * np.exp(2) - a2) * (
        25000 * (a1 * np.exp(2) - a2 * np.exp(x / 10)) +
        2943 * pi * np.exp(1) * (np.exp(x / 10) - 1)
    )

Kontrolle

In [ ]:
print('u( 0) =', u(0))
print('u(20) =', u(20))
print('N( 0) =', N(0))
print('N(20) =', N(20))

Referenzlösungen

In [ ]:
xR = np.linspace(0, p.l, 1000)
uR = u(xR)
nR = N(xR)

Plots

In [ ]:
plt.plot(xR, uR)
plt.title('Displacement $u$')
plt.figure()
plt.plot(xR, nR)
plt.title('Axial force $N$')

## Funktion `pile_fem`

Funktion

In [ ]:
def pile_fem(p, k):

    # TODO: Implementieren

    # Return 
    # return xH, uHat
    return 99

pile_fem(p, 4)

Kontrolle für $N_e = 3$

In [ ]:
xH, uHat = pile_fem(p, 2)
plt.plot(xR, uR, xH, uHat, '-o')
print('uHat = 1e-3 *', 1e3 * uHat, '^T')

## Näherungslösungen für $k = 2, 4, 8, \dots, 256$ Elemente

In [ ]:
plt.plot(xR, uR)

for k in 2**np.arange(1, 9):
    xH, uHat = pile_fem(p, k)
    plt.plot(xH, uHat)

## Fehlerfunktion für $k = 2, 4, 8, \dots, 256$ Elemente

In [ ]:
for k in 2**np.arange(1, 9):
    xH, uHat = pile_fem(p, k)
    e = np.interp(xR, xH, uHat) - uR
    plt.plot(xR, e)

## Fehler für $k = 2, 4, 8, \dots, 8192$ Elemente

In [ ]:
from compute_error import compute_error

es = []
ks = 2**np.arange(1, 14)
hs = p.l / ks

for k in ks:
    xH, uHat = pile_fem(p, k)
    es.append(compute_error(u, xH, uHat))

Fehler in normalem Plot

In [ ]:
plt.plot(hs, es, '-o');

Fehler in loglog-Plot

In [ ]:
plt.plot(hs, es, '-o')
plt.loglog()

Steigung der Geraden im loglog-Plot

In [ ]:
n = (log(es[0]) - log(es[-2])) / (log(hs[0]) - log(hs[-2]))
print('n =', n)

## Normalkraft

In [ ]:
# Exakte Lösung
plt.plot(xR, nR)

# Näherungen
for k in 2**np.arange(1, 9):
    xH, uHat = pile_fem(p, k)
    nH = p.EA * k / p.l * np.diff(uHat)
    nH = np.append(nH[0], nH)
    plt.step(xH, nH)